In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath('../')))

from tqdm import tqdm
import pandas as pd
from utils.file_utils import read_image, read_json, save_image, save_json, read_pkl
from utils.draw_utils import draw_box
from utils.helper_utils import float2_0_1000

# make val

In [5]:
img_root = '/home/shaotao/DATA/AMEX/screenshot'
ann_root = '/home/shaotao/DATA/AMEX/element_anno'
shape_pkl_p = './out/amex_img_shapes.pkl'
val_df_p = './out/amex_val.xlsx'
choice = 'text'
assert choice in {'func', 'text'}
out_json_p = f'amex_val_{choice}.json'

df = pd.read_excel(val_df_p)
# get ori img shape dict
img_shape_dict = read_pkl(shape_pkl_p)
all_ann_p = sorted(os.listdir(ann_root))

INIT_PROMPT = """<image>Output the location of target element according to the given instruction.
## Instruction
{instruction}"""

import random
random.seed(42)
all_datas = []
for img_idx in tqdm(range(len(df))):
    ann_p = os.path.join(ann_root, df.iloc[img_idx]['filename'])
    ann = read_json(ann_p)
    
    img_p = ann['image_path']
    click_ele_lst = ann['clickable_elements']
    h, w = img_shape_dict[img_p]
    img_p = os.path.join(img_root, img_p)
    if img_idx % 100 == 0:
        img = read_image(img_p)
        
    random.shuffle(click_ele_lst)
    conversation = []
    if choice == 'func':
        for ele_idx, ele in enumerate(click_ele_lst):
            box = ele['bbox']
            func_ann = ele.get('functionality', '').strip()
            has_func = func_ann != ''
            if has_func:
                if func_ann.startswith('Click to '):
                    func_ann = func_ann.lower().replace('click to ', '')
                prompt = INIT_PROMPT.format(instruction=func_ann)
                break
            else:
                continue
    else:
        for ele_idx, ele in enumerate(click_ele_lst):
            box = ele['bbox']
            text_ann = ele.get('xml_desc', [])
            has_text = len(text_ann) > 0
            if has_text:
                text_ann = text_ann[0]
                text_ann = text_ann.replace('\n', ',')
                prompt = INIT_PROMPT.format(instruction=text_ann)
                break
            else:
                continue
            
    x1, y1, x2, y2 = box
    gt_box = x1 / w, y1 / h, x2 / w, y2 / h
    if img_idx % 100 == 0:
        img = draw_box(img, gt_box)
        save_image(img, f'tmp_{img_idx}.jpg')
    try:
        gt_box = list(map(float2_0_1000, gt_box))
    except Exception as e:
        print(e)
        continue
    ans = f'({gt_box[0]},{gt_box[1]},{gt_box[2]},{gt_box[3]})'
    conversation.append({'from': 'human', 'value': prompt})
    conversation.append({'from': 'gt', 'value': ans, 'type': 'func' if choice == 0 else 'text'})

    line = {'conversation': conversation, 'image_lst': [os.path.join(img_root, img_p)]}
    all_datas.append(line)
print('total data num: ', len(all_datas))
save_json(all_datas, out_json_p)

100%|██████████| 1000/1000 [00:00<00:00, 2597.33it/s]

total data num:  1000


# make train

In [18]:
img_root = '/home/shaotao/DATA/AMEX/screenshot'
ann_root = '/home/shaotao/DATA/AMEX/element_anno'
shape_pkl_p = './out/amex_img_shapes.pkl'
df_p = './out/amex_func_20k.xlsx'
ele_per_diag = 10
inst_type = 'func'
out_json_p = f'amex_20k_{inst_type}.json'
assert inst_type in ['func', 'text']

df = pd.read_excel(df_p)
# get ori img shape dict
img_shape_dict = read_pkl(shape_pkl_p)

all_ann_p = sorted(os.listdir(ann_root))

INIT_PROMPT = """<image>Output the location of target element according to the given instruction.
## Instruction
{instruction}"""

CONTINUE_PROMPT = """## Instruction
{instruction}"""

import random
random.seed(42)
all_datas = []
for img_idx in tqdm(range(df.shape[0])):
    ann_p = os.path.join(ann_root, df.iloc[img_idx]['filename'])
    ann = read_json(ann_p)
    
    img_p = ann['image_path']
    click_ele_lst = ann['clickable_elements']
    h, w = img_shape_dict[img_p]
    img_p = os.path.join(img_root, img_p)
    if img_idx % 500 == 0:
        img = read_image(img_p)
    random.shuffle(click_ele_lst)
    
    # find specific elements
    final_ele_lst = []
    if inst_type == 'func':
        for ele in click_ele_lst:
            if len(final_ele_lst) == ele_per_diag:
                break
            func_ann = ele.get('functionality', '').strip()
            has_func = func_ann != ''
            if has_func:
                final_ele_lst.append(ele)
    else:
        for ele in click_ele_lst:
            if len(final_ele_lst) == ele_per_diag:
                break
            text_ann = ele.get('xml_desc', [])
            has_text = len(text_ann) > 0
            if has_text:
                final_ele_lst.append(ele)
        
    conversation = []
    for ele_idx, ele in enumerate(final_ele_lst):
        box = ele['bbox']
        if inst_type == 'func':
            func_ann = ele.get('functionality', '').strip()
            has_func = func_ann != ''
            if has_func:
                if func_ann.startswith('Click to '):
                    func_ann = func_ann.lower().replace('click to ', '')
                    
                if ele_idx == 0:   
                    prompt = INIT_PROMPT.format(instruction=func_ann)
                else:
                    prompt = CONTINUE_PROMPT.format(instruction=func_ann)
            else:
                print(f'no func ann in idx: {img_idx}')
                continue
        else:
            text_ann = ele.get('xml_desc', [])
            has_text = len(text_ann) > 0
            if has_text:
                text_ann = text_ann[0]
                text_ann = text_ann.replace('\n', ',')
                if ele_idx == 0:   
                    prompt = INIT_PROMPT.format(instruction=text_ann)
                else:
                    prompt = CONTINUE_PROMPT.format(instruction=text_ann)
            else:
                print(f'no text ann in idx: {img_idx}')
                continue
            
        x1, y1, x2, y2 = box
        x1, y1, x2, y2 = x1 / w, y1 / h, x2 / w, y2 / h
        cent_x, cent_y = (x1 + x2) / 2, (y1 + y2) / 2
        pt = [cent_x, cent_y]
        try:
            pt = list(map(float2_0_1000, pt))
        except Exception as e:
            print('idx: ', img_idx, e)
            continue
        ans = f'({pt[0]},{pt[1]})'
        conversation.append({'from': 'human', 'value': prompt})
        conversation.append({'from': 'gpt', 'value': ans})
        if img_idx % 500 == 0:
            img = draw_box(img, (x1, y1, x2, y2))
    if img_idx % 500 == 0:
        save_image(img, f'tmp_{img_idx}.jpg')
    line = {'conversation': conversation, 'image_lst': [os.path.join(img_root, img_p)]}
    all_datas.append(line)
    if len(all_datas) % 500 == 0:
        if inst_type == 'func':
            print(f'IDX: {len(all_datas)},  sample func_ann: {func_ann}')
        else:
            print(f'IDX: {len(all_datas)},  sample text_ann: {text_ann}')
print('total data num: ', len(all_datas))
save_json(all_datas, out_json_p)

  3%|▎         | 501/20000 [00:00<00:08, 2415.61it/s]

idx:  242 get input: 1.1854636591478696
idx:  242 get input: 1.06265664160401
idx:  242 get input: 1.799498746867168
idx:  242 get input: 1.5538847117794488
idx:  242 get input: 1.431077694235589
idx:  242 get input: 1.6766917293233083
idx:  242 get input: 1.3082706766917294
IDX: 500,  sample func_ann: view details about 'the international americana music show'.


  7%|▋         | 1323/20000 [00:00<00:05, 3196.11it/s]

IDX: 1000,  sample func_ann: enter a destination to search for travel options.
IDX: 1500,  sample func_ann: select economy class for the flight search.


 12%|█▏        | 2494/20000 [00:00<00:04, 3537.06it/s]

...

IDX: 18000,  sample func_ann: open appium settings.
IDX: 18500,  sample func_ann: view or edit the search summary for car rentals in los angeles.


 96%|█████████▌| 19131/20000 [00:05<00:00, 3627.52it/s]

idx:  18786 get input: 1.7392857142857143
idx:  18786 get input: 1.7142857142857142
idx:  18786 get input: 1.8675324675324676
idx:  18786 get input: 1.0061688311688313
idx:  18786 get input: 1.8675324675324676
idx:  18786 get input: 1.3840909090909093
IDX: 19000,  sample func_ann: edit the device name.
idx:  19061 get input: 1.360576923076923
idx:  19061 get input: 1.360576923076923
idx:  19061 get input: 1.360576923076923
idx:  19061 get input: 1.360576923076923
idx:  19061 get input: 1.360576923076923
idx:  19061 get input: 1.360576923076923
idx:  19061 get input: 1.360576923076923
idx:  19061 get input: 1.360576923076923
idx:  19061 get input: 1.360576923076923
idx:  19061 get input: 1.360576923076923
idx:  19199 get input: 1.7301587301587302
idx:  19199 get input: 1.6964285714285716
idx:  19199 get input: 1.7301587301587302
idx:  19199 get input: 1.753968253968254
idx:  19199 get input: 1.3624338624338623
idx:  19199 get input: 1.8662337662337662
IDX: 19500,  sample func_ann: clear

100%|██████████| 20000/20000 [00:05<00:00, 3433.09it/s]


IDX: 20000,  sample func_ann: navigate to the live tab.
total data num:  20000
